In [ ]:
import numpy as np
import tensorflow as tf
import gym
import matplotlib.pyplot as plt
from policy import LQRPolicy as policy
from custompendulumenv import CustomPendulumEnv
from memory import memory
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

Load environment, policy and buffer. Note, the environment returns a cost instead of a reward, meaning that lower values are better

In [ ]:
env = gym.make('CustomPendulum-v0')
pi = policy(env)
buffer = memory()

# Collect data

We do 1000 rollouts a 50 timesteps with the black box policy. Nothing needs to be changed here.

In [ ]:
for ep in range(1000):

    s = env.reset()
    
    for t in range(50):
    
        u = pi.get_action(s) 
            
        s_new, rew, done, _ = env.step(u)
        
        buffer.addTuple([s,u,rew,s_new])
        
        s = s_new

# Fit value function

The value function fitting class. Here, you need to fill in the blank parts

In [ ]:
class ValueFunction:
    def __init__(self, scope='vf'):
        self._scope = scope
        self._sess = tf.get_default_session() or tf.InteractiveSession()
    
        with tf.variable_scope(self._scope, reuse=tf.AUTO_REUSE):
            self._build_graph()
            
        self._sess.run(tf.variables_initializer(self._global_vars()))
    
    def _build_graph(self):
        self.state = tf.placeholder(dtype=tf.float32, shape=(None, 2), name="state")
        self.target = tf.placeholder(dtype=tf.float32, shape=(None), name="target")
        
        # implement neural network here
        # the output of your last layer (size 1) should be stored in self.h
        # Hint: 2 hidden layers a 100 units and relu activations should produce resonable results
        
        self.output = tf.squeeze(self.h)
        
        self.loss = tf.reduce_mean(tf.square(self.output - self.target))
        self.optim = tf.train.AdamOptimizer()
        self.train_op = self.optim.minimize(self.loss)
        
    def _vars(self, scope=''):
        return tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self._scope + '/' + scope)
    
    def _global_vars(self, scope=''):
        return tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self._scope + '/' + scope)
    
    def predict(self, state):
        fddct = {self.state: state}
        return self._sess.run([self.output], feed_dict=fddct)
    
    def fit(self, state, target, iteration=1):
        fddct = {self.state: state, self.target: target}
        for _ in range(iteration):
            _, loss = self._sess.run([self.train_op, self.loss], feed_dict=fddct)
        return loss

In [ ]:
vfunc = ValueFunction()

Fitting routine of the value function. Fill in the blank parts.

In [ ]:
plt_fit = []
iter_fit = 1000
for i in range(iter_fit):

    # sample from the replay buffer
    data=buffer.sample(batch=50)
    xs = np.stack(data[:,0]) # s_t
    xps = np.stack(data[:,3]) # s_t+1
    gs = np.stack(data[:,2]) # rew
    
    vk = vfunc.predict(xs)
    vkplus = vfunc.predict(xps)

    # target
    gamma=0.9
    # implement the fitting target here
    # Hint: td update
    td_target = ...

    # optimize the lsq objective
    fit_loss = vfunc.fit(xs, td_target, iteration=1)
    plt_fit.append(fit_loss)
    
# plot the loss
plt.figure()
plt.plot(range(len(plt_fit)),plt_fit)
plt.draw()

# Visualize

Visualization of the value function. Nothing needs to be changed here.

In [ ]:
def get_obj(x):
    if len(np.asarray(x).shape) < 2:
        x=np.asarray(x).reshape(1,2)
    return np.squeeze(vfunc.predict(x))

In [ ]:
plt.rcParams.update({'font.size': 12})

In [ ]:
xxs =np.linspace(-4,4)
yys =np.linspace(-5,5)

XX,YY=np.meshgrid(xxs,yys)

dots=np.asarray([XX.ravel(),YY.ravel()]).T

scores = get_obj(dots)
scores = scores.reshape(XX.shape)

%matplotlib notebook
fig = plt.figure(figsize=[10,8])
ax = fig.gca(projection='3d')
surf = ax.plot_surface(XX, YY, scores, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax.set_xlabel('angle')
ax.set_ylabel('angle velocity')
ax.set_zlabel('value')

'''
beautiful plot!
'''

# plt.colorbar(cmap=cm.coolwarm)